### Testing the models

In [1]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

import time
from approxeng.input.selectbinder import ControllerResource
from IPython.display import display, clear_output
from jetcam.utils import bgr8_to_jpeg
import numpy as np
import cv2
import ipywidgets
import serial

from keras.models import load_model

# load model. options: PID, DQN, NVIDIA
CHOICE = "NVIDIA"
cv_param = 1
cv_width = 244
cv_height = 244

if(CHOICE == "PID"):
    model = CV_IMG_PROCESSOR()
elif(CHOICE == "DQN"):
    model = load_model('./model/DQN_best.h5')
    cv_param = 0
elif(CHOICE == "NVIDIA"):
    model = load_model('./model/Nvidia_final.h5')
    
# summarize model.
model.summary()

Using TensorFlow backend.
/home/vikram/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vikram/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vikram/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vikram/.local/lib/python3.6/site-packages/tensorflow/python

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 18, 64)         36928     
_____

In [2]:
from camera import Camera
width = int(244)
height = int(244)
camera = Camera(width=width, height=height, capture_width=1280, capture_height=720, capture_fps=60, capture_flip=2)

In [3]:
def getAction():
        img = camera.value
        action = None
        
        if(CHOICE == "PID"):
            action = main_cv.getSteering(img,"canny")
            preview_img = main_cv.preview_img
        elif(CHOICE == "DQN"):
            img2 = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
            preview_img = cv2.resize(img2, (80,80))
            act_values = model.predict(car_agent.conv_to_tensor(preview_img/255))
            action = np.argmax(act_values[0])*10
        elif(CHOICE == "NVIDIA"):
            img2 = cv2.cvtColor(img.copy(), cv2.COLOR_RGB2YUV)
            preview_img = cv2.resize(img2, (200,66))
            img2 = np.expand_dims(preview_img, axis=0)
            action = model.predict(img2/255) 
        else:
            print("ERR")
            action = -1
            
        return action, preview_img

In [4]:
action, _ = getAction()
print(action)

[[86.79137]]


In [26]:
from approxeng.input.selectbinder import ControllerResource
from IPython.display import display, clear_output
import random as rn
import serial
import math

widget_test = ipywidgets.Image(format='jpeg', width=width, height=height/2)
display(widget_test)
RECORD = False
BENCHMARK = True
SHOW_LIVE = True
times = []
Frames = []
speeds = []

# PI Cam 160 FOV 60 FPS @ 1280x720 (GSTREAMER SAYS 120?)
with serial.Serial('/dev/ttyUSB0', 1000000, timeout=1) as ser:
    with ControllerResource() as joystick:
        # Initialize
        STEERING = 90
        THROTTLE = 90
        speed = 0
        ACTIVE = False
        AUTO_THROTTLE = False
        times = []
        
        while joystick.connected:
            if(BENCHMARK):
                start_time = time.time()
                
            if(joystick['cross'] is not None):  # SQUARE EXIT
                print("EXIT")
                ser.close()
                break
                
            if(joystick['triangle'] is not None):  # TRIANGLE TOGGLE AGENT
                if(ACTIVE):
                    print("AGENT OFF")
                    ACTIVE = False   
                else:
                    print("AGENT ACTIVE")
                    ACTIVE = True
                time.sleep(0.2)
                
            if(joystick['square'] is not None):# LB for auto-throttle
                if(AUTO_THROTTLE):
                    print("AUTO_THROTTLE OFF")
                    AUTO_THROTTLE = False
                else:
                    print("AUTO_THROTTLE ON")
                    AUTO_THROTTLE = True
                time.sleep(0.2)
                
            action, preview_img = getAction()
            
            # Default
            THROTTLE = int((joystick['ly']+1)/2*180)
            STEERING = int((joystick['rx']+1)/2*180)
            
            if(ACTIVE):
                STEERING = action
            if(AUTO_THROTTLE):
                THROTTLE = int(-160)
                
            # Manual intervention
            if(joystick['l1'] is not None):# L trigger for manual
                STEERING = int((joystick['rx']+1)/2*180)
                THROTTLE = int((joystick['ly']+1)/2*180)
                
            if(SHOW_LIVE):
                output = "{:05d}-{:05d}\n".format(int(THROTTLE), int(STEERING))
                ser.write(bytes(output,'utf-8'))
                speed_in = (ser.readline())
                if speed_in:
                    speed = int(speed_in)
                    
                info_str = "SPEED {:1d} ACTION {:1d}".format(speed, int(action))
                preview = preview_img
                cv2.putText(preview, info_str, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0))
                widget_test.value = bgr8_to_jpeg(preview)
                
                if(RECORD):
                    Frames.append(preview)
                    speeds.append(speed)

            if(BENCHMARK):
                delta = (time.time() - start_time)
                times.append(delta)
            
if(BENCHMARK):
    mean_delta = np.array(times).mean()
    fps = 1 / mean_delta
    vid_fps = math.ceil(fps)
    print('average(sec):{:.2f}, fps:{:.2f}'.format(mean_delta, fps))

Image(value=b'', format='jpeg', height='122.0', width='200')

AGENT ACTIVE
AUTO_THROTTLE ON
AUTO_THROTTLE OFF
EXIT
average(sec):0.01, fps:68.31


In [24]:
import datetime

x = datetime.datetime.now()
filename = x.strftime("{}_%d-%m-%y_%H:%M.avi").format(CHOICE)
print(filename, vid_fps)

pic_shape = np.shape(preview)
cv_width = pic_shape[1]
cv_height = pic_shape[0]
print(cv_width, cv_height)

preview_widget = ipywidgets.Image(format='jpeg', width=width, height=height/2)
display(preview_widget)

out = cv2.VideoWriter(filename,cv2.VideoWriter_fourcc(*"MJPG"), vid_fps,(cv_width,cv_height), cv_param)
for f in Frames:
    out.write(f.astype('uint8'))
    preview_widget.value = bgr8_to_jpeg(f)
    
out.release()

NVIDIA_18-12-19_01:46.avi 34
200 66


Image(value=b'', format='jpeg', height='122.0', width='200')

In [ ]:
camera.unobserve_all() 
camera.stop()